In [1]:
function modifiedMidpointRule(f,tspan,z0,n)
    #[t,z] = modifiedMidpointRule(f,tspan,x0,n)
    #
    #Approximates the solution to the initial value problem
    #by numerical integration with the modified midpoint rule
    #
    nt = n+1
    nz = size(z0,1)
    
    t0 = tspan[1]
    t1 = tspan[2]
    h = (t1-t0)/n
    t = linspace(tspan1, tspan2, nt)
    z = zeros(nz,n)
    
    #initialize and then run the modified mid-point method
    z[:,1] = z0
    z[:,2] = z0 + h*f(t0,z0)
    for i=3:nt
        z[:,i] = z[:,1-2] + 2*h*f(t[i-1],z[:,i-1])
    end
    
    #Refine the final point using the dynamics function 
    #at the final point
    z[:,nt] = 0.5*(z[:,nt] + z[:,nt-1] + h*f(t[nt],z[:,nt]))
    
    t,z
end
function BulirschStoerStep(f, tspan, z0, tol)
    #[zF, info] = BulirschStoerStep(f,tspan,z0,tol)
    #
    #Computes a single step using Bulirsch Stoer Method
    #
    
    #Set an upper limit ont he number of mesh refinements
    #in the sequence
    nRefineMax = 8
    
    #Simple Logistics and Memory allocation
    n = 2*(1:nRefineMax)
    nz = size(z0,1)
    if length(tol)==1
        tol = tol*ones(size(z0))
    end
    T = zeros(nz,nRefineMax,nRefineMax) #Extrapolation table
    E = zeros(nz,nRefineMax) #Error estimate table
    
    #assume we fail to meet tolerance
    info.exit = "maxRefine"
    for j=1:nRefineMax #loop over the sequence of improving meshs
        
        #compute the estimate of the solution on the current path
        z = modifiedMidpointRule(f,tspan,z0,n[j])
        T[:,j,1] = z[:,end]
        
        if j>1
            
            #Compute the extrapolation table series
            for k=2:j
                num = T(:,j,k-1) - T(:,j-1,k-1)
                den = (n(j)/(n(j-k+1)))^2 - 1
                T[:,j,k] = T[:,j,k-1] + num/den
            end
            
            #Compute the error estimates:
            E[:,j] = abs(T[:,j,j-1] - T[:,j,j])
            
            #Check convergence
            if all(E[:,j]<tol)
                #research IJulia field access!!!
                info.exit = "converged"
                break
            end
        end
    end
    
    #Other useful things
    info.error = E[:,j]
    info.nFunEval = sum(n[1:j])
    info.nRefine = j
    
    #Return the Estimate of the solution
    zF = T(:,j,j)
    
    zF,info
end

BulirschStoerStep (generic function with 1 method)

In [2]:
function BulirschStoer(f,t,z0,tol)
    #[z,info] = BulirschStoer(f,t,z0,tol)
    #
    #Solves an IVP using the BulirschStoer method This method is 
    #ideal for high-accuracy solutions to smooth IVP
    #
    #Computes z(t) such that dz/dt = f(t,z) starting from the 
    #initial state z0. The sol at the grid points will be 
    #accurate within tol
    #
    #If the provided grid is insufficient, this function will
    #automatically introduce intermediate grid points to acheive
    #required accuracy
    #
    
    #if a step fails how many substeps should be created in its
    #place?
    nStepRefine = 3
    
    #Logistics and memory allocation:
    t = copy(t)
    z0 = copy(z0)
    tol = copy(tol)
    nt = length(t)
    nz = size(z0,1)
    z = zeros(nz,nt)
    z[:,1] = z0
    info.error = zeros(size(z))
    info.nFunEval = zeros(1,nt)
    
    #March forward in time from grid point to grid point
    for i=2:nt
        
        tspan = [t[i-1], t[i]]
        zF,stepInfo = BulirschStoerStep(f,tspan,z(:,i-1),tol)
        
        if stepInfo.exit=="converged"#successful step
            z[:,i] = zF
            info.error[:,i] = stepInfo.error
            info.nFunEval[i] = stepInfo.nFunEval
            
        else
            time = linspace(tspan[1],tspan[2],nStepRefine+1)
            zTmp,infoTmp = BulirschStoer(f,time,z[:,i-1],tol)
            z[:,i] = zTmp[:,end]
            info.error[:,i] = infoTmp.error[:,end]
            info.nFunEval[i] = sum(infoTmp.nFunEval)
            
        end
    end
    z,info
end
            

BulirschStoer (generic function with 1 method)

In [5]:
t = [1,6]
z0 = [1,1]
tol = 1e-8
z,info = BulirschStoer(f,t,z0,tol)

LoadError: [91mtype #info has no field error[39m

In [4]:
function f(t,x)
    [x[1]^2+x[2]^2-1; x[2]-x[1]]
end

f (generic function with 1 method)

In [19]:
struct Foo
    bar
    baz
end
foo = Foo("converged",2)
foo.bar = "diverged"

LoadError: [91mtype Foo is immutable[39m

In [27]:
x0 = [1,1,1]
kk=[2,2,222]
xx=zeros(length(x0),length(kk))
gg = ones(size(xx))

3×3 Array{Float64,2}:
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0

In [12]:
t = [1,1,1,1]
nt = length(t)

4